<a href="https://colab.research.google.com/github/nithin-k-mundrathi/LLM-practice/blob/main/RAG/RAG_C_2/3_Embedding_Based_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
try:
  import deeplake
except:
  !pip install deeplake==3.9.18
  import deeplake

In [1]:

#Google Drive option to store API Keys
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

# For Google Colab and Activeloop while waiting for Activeloop (April 2024) pending new version
#This line writes the string "nameserver 8.8.8.8" to the file. This is specifying that the DNS server the system
#should use is at the IP address 8.8.8.8, which is one of Google's Public DNS servers.
with open('/etc/resolv.conf', 'w') as file:
   file.write("nameserver 8.8.8.8")

In [7]:
#Retrieving and setting the Activeloop API token
f = open("activeloop.txt", "r")
API_token=f.readline().strip()
f.close()
ACTIVELOOP_TOKEN=API_token
os.environ['ACTIVELOOP_TOKEN'] =ACTIVELOOP_TOKEN

In [54]:
%%capture
!pip install sentence-transformers==3.0.1

In [ ]:
from sentence_transformers import SentenceTransformer

In [25]:
vector_store_path = "hub://nithinkmundrathi/space_exploration_v2"

In [26]:
from deeplake.core.vectorstore.deeplake_vectorstore import VectorStore
import deeplake.util
ds = deeplake.load(vector_store_path)

vector_store = VectorStore(path=vector_store_path)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/nithinkmundrathi/space_exploration_v2



/usr/local/lib/python3.10/dist-packages/deeplake/util/warnings.py:7: UserWarning: Checking out dataset in read only mode as another machine has locked this version for writing.
  warnings.warn(*args, **kwargs)

|

hub://nithinkmundrathi/space_exploration_v2 loaded successfully.



Deep Lake Dataset in hub://nithinkmundrathi/space_exploration_v2 already exists, loading from the storage


In [27]:
ds.size_approx()

58000000

## Retrieval Query

In [34]:
def embedding_function(texts, model=model_sentence):
   if isinstance(texts, str):
       texts = [texts]
   texts = [t.replace("\n", " ") for t in texts]
   return [model.encode(data) for data in texts]

In [35]:
def get_user_prompt():
    # Request user input for the search prompt
    return input("Enter your search query: ")

def search_query(prompt):
    # Assuming `vector_store` and `embedding_function` are already defined
    search_results = vector_store.search(embedding_data=prompt, embedding_function=embedding_function)
    return search_results

In [36]:
# Get the user's search query
#user_prompt = get_user_prompt()
# or enter prompt if it is in a queue
user_prompt="Tell me about space exploration on the Moon and Mars."

# Perform the search
search_results = search_query(user_prompt)

In [37]:
# Print the search results
print(len(search_results))

4


In [38]:
search_results

{'id': ['84a0c012-cd58-11ef-bd7a-0242ac1c000c',
  '84a0c648-cd58-11ef-bd7a-0242ac1c000c',
  '84a0b3ba-cd58-11ef-bd7a-0242ac1c000c',
  '84a0ce4a-cd58-11ef-bd7a-0242ac1c000c'],
 'metadata': [{'source': 'llm.txt'},
  {'source': 'llm.txt'},
  {'source': 'llm.txt'},
  {'source': 'llm.txt'}],
 'text': ['rbiters, and in 2008 the Indian Moon Impact Probe and in 2023 the Chandrayaan-3 of India became the first spacecraft to land on the lunar south pole. Crewed exploration of the Moon began in 1968 with the Apollo 8 mission that successfully orbited the Moon, the first time any extraterrestrial object was orbited by humans. In 1969, the Apollo 11 mission marked the first time humans set foot upon another world. Crewed exploration of the Moon did not continue for long. The Apollo 17 mission in 1972 marked the sixth landing and the most recent human visit. Artemis 2 is scheduled to complete a crewed flyby of the Moon in 2025, and Artemis 3 will perform the first lunar landing since Apollo 17 with 

In [39]:
# Function to wrap text to a specified width
def wrap_text(text, width=80):
    lines = []
    while len(text) > width:
        split_index = text.rfind(' ', 0, width)
        if split_index == -1:
            split_index = width
        lines.append(text[:split_index])
        text = text[split_index:].strip()
    lines.append(text)
    return '\n'.join(lines)

In [40]:
import textwrap

# Assuming the search results are ordered with the top result first
top_score = search_results['score'][0]
top_text = search_results['text'][0].strip()
top_metadata = search_results['metadata'][0]['source']

# Print the top search result
print("Top Search Result:")
print(f"Score: {top_score}")
print(f"Source: {top_metadata}")
print("Text:")
print(wrap_text(top_text))

Top Search Result:
Score: 0.6055324673652649
Source: llm.txt
Text:
rbiters, and in 2008 the Indian Moon Impact Probe and in 2023 the Chandrayaan-3
of India became the first spacecraft to land on the lunar south pole. Crewed
exploration of the Moon began in 1968 with the Apollo 8 mission that
successfully orbited the Moon, the first time any extraterrestrial object was
orbited by humans. In 1969, the Apollo 11 mission marked the first time humans
set foot upon another world. Crewed exploration of the Moon did not continue
for long. The Apollo 17 mission in 1972 marked the sixth landing and the most
recent human visit. Artemis 2 is scheduled to complete a crewed flyby of the
Moon in 2025, and Artemis 3 will perform the first lunar landing since Apollo
17 with it scheduled for launch no earlier than 2026. Robotic missions are
still pursued vigorously. Mars [ edit ] Main article: Exploration of Mars
Surface of Mars by the Spirit rover (2004) The exploration of Mars has been an
important pa

##Augmented Input

In [42]:
augmented_input=user_prompt+" "+top_text

In [43]:
print(augmented_input)

Tell me about space exploration on the Moon and Mars. rbiters, and in 2008 the Indian Moon Impact Probe and in 2023 the Chandrayaan-3 of India became the first spacecraft to land on the lunar south pole. Crewed exploration of the Moon began in 1968 with the Apollo 8 mission that successfully orbited the Moon, the first time any extraterrestrial object was orbited by humans. In 1969, the Apollo 11 mission marked the first time humans set foot upon another world. Crewed exploration of the Moon did not continue for long. The Apollo 17 mission in 1972 marked the sixth landing and the most recent human visit. Artemis 2 is scheduled to complete a crewed flyby of the Moon in 2025, and Artemis 3 will perform the first lunar landing since Apollo 17 with it scheduled for launch no earlier than 2026. Robotic missions are still pursued vigorously. Mars [ edit ] Main article: Exploration of Mars Surface of Mars by the Spirit rover (2004) The exploration of Mars has been an important part of the spa

In [44]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [46]:
f = open("/content/open-aikey.txt", "r")
API_KEY=f.readline().strip()
f.close()
#The OpenAI Key
import os
import openai
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [47]:
import openai
from openai import OpenAI
import time

client = OpenAI()
gpt_model="gpt-4o-mini"
start_time = time.time()  # Start timing before the request

def call_gpt4_with_full_text(itext):
    # Join all lines to form a single string
    text_input = '\n'.join(itext)
    prompt = f"Please summarize or elaborate on the following content:\n{text_input}"

    try:
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {"role": "system", "content": "You are a space exploration expert."},
                {"role": "assistant", "content": "You can read the input and answer in detail."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1  # Fine-tune parameters as needed
        )
        return response.choices[0].message.content
    except Exception as e:
        return str(e)

gpt4_response = call_gpt4_with_full_text(augmented_input)

response_time = time.time() - start_time  # Measure response time
print(f"Response Time: {response_time:.2f} seconds")  # Print response time

print(gpt_model, "Response:", gpt4_response)

Response Time: 9.20 seconds
gpt-4o-mini Response: The content discusses significant milestones in space exploration, particularly focusing on the Moon and Mars. Here’s a detailed summary:

### Moon Exploration

1. **Early Missions**:
   - The exploration of the Moon began with the **Apollo program** in the late 1960s. 
   - **Apollo 8** (1968) was a landmark mission as it successfully orbited the Moon, marking the first time humans orbited another celestial body.
   - **Apollo 11** (1969) was historic for being the first mission where humans set foot on the Moon, with astronauts Neil Armstrong and Buzz Aldrin landing on the lunar surface.

2. **Subsequent Missions**:
   - The Apollo program continued with several missions, with **Apollo 17** (1972) being the last crewed mission to the Moon, marking the sixth lunar landing and the most recent human visit to the Moon.

3. **Recent Developments**:
   - In 2008, India launched the **Indian Moon Impact Probe**, contributing to lunar explora

## Evaluation

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])
    return similarity[0][0]

similarity_score = calculate_cosine_similarity(user_prompt, gpt4_response)

print(f"Cosine Similarity Score: {similarity_score:.3f}")

Cosine Similarity Score: 0.422


In [49]:
similarity_score = calculate_cosine_similarity(augmented_input, gpt4_response)

print(f"Cosine Similarity Score: {similarity_score:.3f}")

Cosine Similarity Score: 0.852


In [53]:
def calculate_cosine_similarity_with_embeddings(text1, text2):
    embeddings1 = model_sentence.encode(text1)
    embeddings2 = model_sentence.encode(text2)
    similarity = cosine_similarity([embeddings1], [embeddings2])
    return similarity[0][0]

similarity_score = calculate_cosine_similarity_with_embeddings(augmented_input, gpt4_response)
print(f"Cosine Similarity Score: {similarity_score:.3f}")

Cosine Similarity Score: 0.839
